In [ ]:
import mne
import numpy as np
import os
from matplotlib import pyplot as plt
from mne.preprocessing import ICA
import pandas as pd
from mne.minimum_norm import apply_inverse, make_inverse_operator



# Introduction

In this tutorial, you will learn how to analyze data from an MEG study involving two blocks. The workflow is applicable to studies with one block, or 3+, as well. Basically, you need to perform preprocessing separately on every continuous block of data that you collect, and the way that you preprocess each block should be kept consistent to allow for comparisons across blocks.

In this tutorial, one block is a primed word-reading task (comp), and the other is a primed picture-naming task (prod). This means that there are two experimental files, which need to be preprocessed equivalently so that they may be compared to one another.

![trial](Trial.jpg "Trial")

# Custom functions

Specific to the UMD MEG system, it is necessary to manually correct the location of (0-indexed) MEG sensor 56. The .fif file type has an attribute which identifies the locations of every channel, indexed via *raw.info['chs'][idx]['loc']*, where idx is the index of a channel whose location you want to view. It's important that the location of all channels is correct for many parts of the preprocessing pipeline, like interpolating bad channels.

We have the location value that MEG 056 *should* have. The function below will correct the sensor location and should be called for all files recorded using the UMD MEG system.

In [ ]:
def fix_56(raw):
    # Fix the location of MEG 056, necessary for all data collected at KIT-UMD MEG Lab
    loc = np.array([
        0.09603, -0.07437, 0.00905, -0.5447052, -0.83848277,
        0.01558496, 0., -0.01858388, -0.9998273, 0.8386276,
        -0.54461113, 0.01012274])
    index = raw.ch_names.index('MEG 056')
    raw.info['chs'][index]['loc'] = loc
    return raw

# Importing Files

Change the ROOT filepath to the directory that the tutorial is stored in on your computer. Everything else *should* be fine to run without editing, save for the ICA components to remove.

In [ ]:
sub = 'R3289'
condition = 'G'

In [ ]:
ROOT = f"/Users/audreylaun/Library/CloudStorage/Box-Box/UMD_MEG_Admin_Only/Analysis/"
input_dir = f"{ROOT}Tutorial/{sub}/"
output_dir = f"{ROOT}Tutorial/{sub}/Output"

For this experiment, it's necessary to store the conditions for the comprehension and production blocks, these can be obtained based on the overall condition value (A-H)

In [ ]:
comp_condition = 'X'
prod_condition = 'X'
if condition in ['A', 'D']:
    comp_condition = 'A1'
    prod_condition = 'B1'
elif condition in ['B','C']:
    comp_condition = 'B1'
    prod_condition = 'A1'
elif condition in ['E','H']:
    comp_condition = 'A2'
    prod_condition = 'B2'
elif condition in ['F', 'G']:
    comp_condition = 'B2'
    prod_condition = 'A2'

We will load experimental files (comp-raw and prod-raw) *and* empty room files (emptyroom). The empty room file can be used for denoising, or noise covariance. In this specific tutorial, we do not use the empty room. Nevertheless, it is still good practice to preprocess the empty room file the exact same way that you preprocess your experimental data, in case it becomes useful down the line.

The experimental files loaded here have already been converted from .sqd to .fif using the kit2fiff terminal GUI.

The empty room is still a sqd file. The conversion is not necessary because we do not need to set the head shape/marker measurement attributes because there are none for these data. Instead, we will assign the info from an experimental fif file so that we can Maxwell filter the empty room file the same way as the experimental blocks.

In [ ]:
comp_raw_fname = input_dir + sub + '_comp-raw.fif'
prod_raw_fname = input_dir + sub + '_prod-raw.fif'
empty_room_fname = input_dir + '/DAQ/' + sub + '_emptyroom.sqd'

comp_raw = mne.io.read_raw_fif(comp_raw_fname, preload=True)
prod_raw = mne.io.read_raw_fif(prod_raw_fname, preload=True)
empty_room_raw = mne.io.read_raw_kit(empty_room_fname, preload=True)
empty_room_raw = mne.io.RawArray(empty_room_raw.get_data(), comp_raw.info)

# Fix the location of sensor 56

We call our custom function is called to fix the location for all files.

In [ ]:
comp_raw = fix_56(comp_raw)
prod_raw = fix_56(prod_raw)
empty_room_raw = fix_56(empty_room_raw)

# Excluding bad channels

MEG 056 and MEG 086 are always bad. 056 shows up inside the head and, even though the position can be corrected, the measurements it takes are not trustworthy. 086 is flat. We will always mark these as bad

In [ ]:
bads = ['MEG 056', 'MEG 086']

Other sensors are *sometimes* bad. We will call the [mne.preprocessing.find_bad_channels_maxwell_](https://mne.tools/stable/generated/mne.preprocessing.find_bad_channels_maxwell.html) function to automatically detect these channels

We will look for bad channels in both experimental files, creating a "master list" of bads, which will be excluded from both experimental files and the empty room.

In [ ]:
noisy_chs, flat_chs = mne.preprocessing.find_bad_channels_maxwell(
        comp_raw, ignore_ref=True
    )
for i in bads:
    if i not in noisy_chs:
        noisy_chs.append(i)
noisy_chs, flat_chs = mne.preprocessing.find_bad_channels_maxwell(
        prod_raw, ignore_ref=True
    )
for i in bads:
    if i not in noisy_chs:
        noisy_chs.append(i)


comp_raw.info['bads'] = noisy_chs
prod_raw.info['bads'] = noisy_chs
empty_room_raw.info['bads'] = noisy_chs

# Maxwell Filtering

We will use the [mne.preprocessing.maxwell_filter](https://mne.tools/stable/generated/mne.preprocessing.maxwell_filter.html#mne.preprocessing.maxwell_filter) function to filter the data using mutipole moments. Essentially, this preprocessing step will clean the data by suppressing signals from distant sources.

Because of our specific MEG system, it is important to specifically set some parameters for this function. Specifically, we need to set **ignore_ref** to **True** and **st_only** to **True**. This means that we will only perform temporal projections (tSSS instead of SSS) on the output data because our the KIT system does not allow for cross-talk cancellation and movement compensation.

In [ ]:
comp_raw_tsss = mne.preprocessing.maxwell_filter(
        comp_raw,
        st_duration=10,
        ignore_ref=True,
        st_correlation=0.9,
        st_only=True
    )

prod_raw_tsss = mne.preprocessing.maxwell_filter(
    prod_raw,
    st_duration=10,
    ignore_ref=True,
    st_correlation=0.9,
    st_only=True
)

empty_room_raw_tsss = mne.preprocessing.maxwell_filter(
    empty_room_raw,
    st_duration=10,
    ignore_ref=True,
    st_correlation=0.9,
    st_only=True
)

os.makedirs(output_dir, exist_ok=True)
fname = output_dir + '/' + sub + '_emptyroom_preproc-raw.fif'
# mne.io.Raw.save(empty_room_raw_tsss, fname, overwrite=True)

# ICA

We will now perform ICA on the data. The purpose of ICA is to remove muscular artifacts from the MEG data. These artifacts are eye blinks, heart beats, and saccades. Therefore, we will not run ICA on our empty room data.

We will fit our ICA model ot a 1Hz high-pass filtered version of the data to eliminate any slow drifts that would decrease the calculated independence of sources.

To carry out ICA, the MNE function [mne.preprocessing.ICA](https://mne.tools/stable/generated/mne.preprocessing.ICA.html) first conducts a principal components analysis (PCA) to whiten the data. The resulting PCA components explain 99% of the variance. Then, it will pass the first 30 components to the ICA algorithm. We will fit the model using the fastica method, which is default.

Below are examples of each type of artifact as seen on the topographic and timecourse plots.
![artifacts](Artifacts.jpg "Artifacts")



We will start with the comprehension. ICA is the one part of preprocessing that needs to be done slightly differently for the two blocks, because different component numbers will refer to artifacts.

In [ ]:
comp_raw_tsss_filt = comp_raw_tsss.copy().filter(l_freq=1.0, h_freq=None)
ica = ICA(n_components=30, max_iter="auto")
ica.fit(comp_raw_tsss_filt)

ica.plot_sources(comp_raw_tsss, show_scrollbars=True)
ica.plot_components()

From the topographic maps and timecourse plots, you will need to manually identify components that contain eye blinks, saccades, or heartbeats. Try to not exclude anything that you can't attribute to one of these muscular artifacts.

Based on the outputs, **enter the components to be excluded below**

In [ ]:
ica_excluded = [0,1,4,12]

Then, apply the ICA model to the raw data

In [ ]:
ica.exclude = ica_excluded
ica.apply(comp_raw_tsss)

We can now [interpolate](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.interpolate_bads) the values for the bad channels, using values from neighboring sensors to infer the activity that *would* have been in that sensor if it were functioning properly

In [ ]:
comp_raw_tsss = comp_raw_tsss.interpolate_bads()

Now, we can save the "cleaned" continuous file.

In [ ]:
fname = output_dir + '/' + sub + '_comp_preproc-raw.fif'
mne.io.Raw.save(comp_raw_tsss, fname, overwrite=True)

We will do the same for the production block.

In [ ]:
# High pass filter at 1Hz
prod_raw_tsss_filt = prod_raw_tsss.copy().filter(l_freq=1.0, h_freq=None)

#Fit ICA model with 30 components
ica = ICA(n_components=30, max_iter="auto")
ica.fit(prod_raw_tsss_filt)

#Plot ICA component scalp topographies and timecourses
ica.plot_sources(prod_raw_tsss, show_scrollbars=True)
ica.plot_components()

**Enter the components to exclude**

In [ ]:
ica_excluded = [0,1,2,3,9]

In [ ]:
ica.exclude = ica_excluded
ica.apply(prod_raw_tsss)

In [ ]:
prod_raw_tsss = prod_raw_tsss.interpolate_bads()

In [ ]:
fname = output_dir + '/' + sub + '_prod_preproc-raw.fif'
mne.io.Raw.save(prod_raw_tsss, fname, overwrite=True)

The continuous data should now look much "cleaner" that it did originally. Let's take a look at the raw and preprocessed files.

In [ ]:
comp_raw.plot()

In [ ]:
comp_raw_tsss.plot()

# Create Epoch objects for each block

We will now find events that are indicated by trigger channels. We will use separate dictionaries for comprehension and production.

The keys in the event dictionary are the names that you would like to refer to the events as, and the values are the trigger channel number. The trigger channel values are set by the researcher in the experimental script.

It is best practice to include a text file that keeps track of these values. The text files for this experiment are within the parent folder, and the values are as follows:

**Comprehension**\
*Practice*\
137 - Audio onset\
138 - audio offset\
139 - Word \
? Word identical to prime? \
140 - yes comprehension question \
141 - no comprehension question

*Experiment*\
130 - Word Identical to prime (50x)\
132 - Word unrelated to prime (50x)\
136 - Picture unrelated in both lists (50x)\
134 — Audio onset (150x)\
135 — Audio offset (150x)

Buttons\
142 - LEFT\
143 - RIGHT

Questions:\
Yes - 146\
No - 147

**Production**\
*Practice* \
137 - Audio onset \
138 - audio offset\
139 - picture identical to prime \
140 - picture unrelated to prime

Experiment \
130 - Picture Identical to prime (50x)\
132 - Word unrelated to prime (50x)\
136 - Picture unrelated in both lists (50x) \
131 - Production trigger (fires after 130/132) \
134 — Audio onset (150x)\
135 — Audio offset (150x)

Buttons\
142 - LEFT \
143 - RIGHT

We will now define our events dictionary for the comprehension condition. We are only going to be analyzing the target onsets, so our dictionary will only include those event codes.

In the production block, the data are epoched based on the onset of the target image. In the comprehension block, the data are epoched based on the onset of the target word. In other words, when looking at evoked responses, t=0 represents the onset of the image/word.

We will keep the ignore condition (words that were never targets in any counterbalancing, or, fillers) for reasons that will be clear later.

In [ ]:
event_dict = {
    "comprehension identical": 162,
    "comprehension unrelated": 164,
    "ignore": 168
}

You may notice that the keys in this dictionary do not match the values from the text file. This is because the value that you input into your script the "binary" trigger value, while the values contained in the .sqd file is the "MEG160" trigger value. Below is the conversion table between binary and MEG160 (hint, just add 32)

![conversions](trigger_conversions.png "Trigger Conversions")

We will now use our event dictionary to "label" trials of interest and then save the data as an [epoch object](https://mne.tools/stable/generated/mne.Epochs.html).


In [ ]:
# Comprehension
events = mne.find_events(comp_raw_tsss, stim_channel="STI 014")

epochs_comp = mne.Epochs(comp_raw_tsss, events, tmin=-0.3, tmax=0.6, event_id=event_dict, preload=True)

We also don't want any trials where the maximum peak amplitudes exceed a threshold, which would suggest that a movement artifact, power surge, or some other impurity pervaded in the data through our preprocessing, so we will also excludeepochs that have a max peak to peak signal amplitude that exceeds 3 picoteslas.

In [ ]:
reject_criteria = dict(mag=3000e-15)
epochs_comp.drop_bad(reject=reject_criteria)

We will now save our comprehension epoch object

In [ ]:
fname = output_dir + '/' + sub + '_comp-epo.fif'
# epochs_comp.save(fname, overwrite=True)

We will do the same thing with the production block.

In [ ]:
events = mne.find_events(prod_raw_tsss, stim_channel="STI 014")

event_dict = {
    "production identical": 162,
    "production unrelated": 164,
    "ignore": 168
}
epochs_prod = mne.Epochs(prod_raw_tsss, events, tmin=-0.3, tmax=0.7, event_id=event_dict, preload=True)

Instead of saving this epoch object immediately, we will **exclude trials that contain bad productions** (where the participant did not name the object correctly).

The production data are stored in the .xlsx file "Productions_A2" in the tutorial folder. As a refresher, A2 is the production condition in the "G" counterbalancing for the experiment.

In this excel sheet, we have information about the intended word, the trigger code, and what the participant produced. With this informaiton, we know if the participant was correct, and if the word had been repetition primed or not.

In [ ]:
excel_fname = f"{ROOT}Tutorial/Productions_{prod_condition}.xlsx"
df = pd.read_excel(excel_fname)
column_name = sub + ' Accuracy'
bad_productions = df.index[df[column_name] == False].tolist()

This is the part where it matters that we kept the "ignore" condition. The way that we will exclude the bad productions is by using the index of that trial from the excel sheet. The excel sheet contains all words, those that were repetition primed, unrelated primed, and filler words (150 words). If we did not keep the "ignore" condition in our events dictionary, then we would only have 100 items in our epochs object and we would not be able to exclude trials using the indeces from the excel sheet.

Let's look at our epochs object containing **all** productions

In [ ]:
print(epochs_prod)
len(epochs_prod)

Now, let's drop the bad productions

In [ ]:
epochs_prod.drop(bad_productions)

Let's look at our epoch object again.

In [ ]:
print(epochs_prod)
len(epochs_prod)

We can now see that this particular participant had 30 incorrect productions total. 15 of those bad productions were for filler words and the other 15 were for unrelated primed words. All 50 identically primed words were produced correctly.

We will again reject bad epochs that have a max peak to peak signal amplitude that exceeds 3 picoteslas. It's important to do this step **after** excluding the bad productions to ensure that the indexing works properly.

In [ ]:
reject_criteria = dict(mag=3000e-15)
epochs_prod.drop_bad(reject=reject_criteria)

We will now save the production epochs.


In [ ]:
fname = output_dir + '/' + sub + '_prod-epo.fif'
# epochs_prod.save(fname, overwrite=True)

# Equalizing epoch counts

We don't want to have uneven counts for each condition (production identical, production unrelated, comprehension identical, comprehension unrelated) when we are doing data analysis, because higher power for some conditions will bias the source estimates.

 We will equalize epoch counts by creating separate objects for each condition and then randomly excluding trials from objects that have a larger epoch count than the minimum, until they are all the same. In this subject, the lowest epoch count was in the production unrelated condition, so after this function is called, the other three conditions will have the same number of epochs as that condition.

In [ ]:
ident_prod = epochs_prod['production identical'].pick('mag')
unrel_prod = epochs_prod['production unrelated'].pick('mag')
ident_comp = epochs_comp['comprehension identical'].pick('mag')
unrel_comp = epochs_comp['comprehension unrelated'].pick('mag')
mne.epochs.equalize_epoch_counts([ident_prod, ident_comp, unrel_prod, unrel_comp], method="random")

# Create evoked objects for each condition

Now that the epoch counts are equal, we can make evoked objects for each condition, which contain the average response for each condition. We low pass filter the data at 40 Hz because any frequency higher than that is not relevant to the study.

In [ ]:
ident_prod = ident_prod.average().filter(l_freq=None, h_freq=40)
unrel_prod = unrel_prod.average().filter(l_freq=None, h_freq=40)
ident_comp = ident_comp.average().filter(l_freq=None, h_freq=40)
unrel_comp = unrel_comp.average().filter(l_freq=None, h_freq=40)

# Plotting your evoked data

We can take a look at our evoked data by plotting the timecourse and topography of different conditions.

In [ ]:
evokeds_dict = {
    "Prod Ident": ident_prod,
    "Prod Unrel": unrel_prod,
    "Comp Ident": ident_comp,
    "Comp Unrel": unrel_comp,
}

## All sensors timecourse

In [ ]:
#Timecourse over all sensors
mne.viz.plot_compare_evokeds(evokeds_dict, picks='mag', colors=['red', 'red', 'blue', 'blue'], linestyles=['-', '--', '-', '--'])

## Subset of sensors timecourse

In [ ]:
# Get subsets of sensors
# left posterior
left_post_numbers = [4, 5, 6, 7, 8, 9, 34, 36, 37, 38, 40, 47, 48, 49, 50, 75, 76, 77, 79, 88, 127, 129,
                     137, 89, 92, 94, 12, 10, 11, 35, 46, 51, 72, 73, 74, 78, 91, 125, 126, 138, 140, 141, 128, 41]
left_post = []
for i in left_post_numbers:
    title = ""
    if i < 10:
        title = 'MEG 00' + str(i)
    elif 10 <= i < 100:
        title = 'MEG 0' + str(i)
    else:
        title = 'MEG ' + str(i)
    left_post.append(title)

#left anterior
left_ant_numbers = [1, 2, 3, 39, 42, 43, 44, 80, 81, 86, 83, 84, 85, 108, 130, 131, 132, 133, 134, 135,
                    136, 151, 65, 59, 152, 53, 68, 143, 105, 106, 107, 109, 45, 111]
left_ant = []
for i in left_ant_numbers:
    title = ""
    if i < 10:
        title = 'MEG 00' + str(i)
    elif 10 <= i < 100:
        title = 'MEG 0' + str(i)
    else:
        title = 'MEG ' + str(i)
    left_ant.append(title)
print(left_ant)

#right posterior
right_post_numbers = [14, 15, 16, 17, 18, 19, 27, 28, 30, 54, 56, 57, 66, 69, 70, 97, 119, 121, 122,
                      90, 87, 71, 52, 82, 58, 67, 95, 26, 145, 13, 29, 31, 32, 33, 120, 123, 124, 142]
right_post = []
for i in right_post_numbers:
    title = ""
    if i < 10:
        title = 'MEG 00' + str(i)
    elif 10 <= i < 100:
        title = 'MEG 0' + str(i)
    else:
        title = 'MEG ' + str(i)
    right_post.append(title)

#right anterior
right_ant_numbers = [20, 21, 22, 23, 24, 60, 61, 63, 99, 100, 114, 115, 116, 117, 118, 147,
                     148, 155, 96, 25, 62, 64, 98, 101, 102, 103, 104, 112, 113, 119, 93]
right_ant = []
for i in right_ant_numbers:
    title = ""
    if i < 10:
        title = 'MEG 00' + str(i)
    elif 10 <= i < 100:
        title = 'MEG 0' + str(i)
    else:
        title = 'MEG ' + str(i)
    right_ant.append(title)

In [ ]:
# Plot subsets of sensors
fig, axes = plt.subplots(2, 2, figsize=(13, 10))
fig.tight_layout(pad=4.0)

ylim = dict(mag=[0, 45])
# Top-left
mne.viz.plot_compare_evokeds(
    evokeds_dict,
    picks=left_post,
    axes=axes[1, 0],
    colors=['red', 'red','blue', 'blue'], linestyles=['-', '--', '-', '--'],
    show=False,
    ylim=ylim
)
axes[1, 0].set_title("Left Posterior",size=20)

# Top-right
mne.viz.plot_compare_evokeds(
    evokeds_dict,
    picks=left_ant,
    axes=axes[0, 0],
    colors=['red', 'red','blue', 'blue'], linestyles=['-', '--', '-', '--'],
    show=False,
    ylim=ylim
)
axes[0, 0].set_title("Left Anterior",size=20)

# Bottom-left
mne.viz.plot_compare_evokeds(
    evokeds_dict,
    picks=right_post,
    axes=axes[1, 1],
    colors=['red', 'red','blue', 'blue'], linestyles=['-', '--', '-', '--'],
    show=False,
    ylim=ylim
)
axes[1, 1].set_title("Right Posterior", size=20)

# Bottom-right
mne.viz.plot_compare_evokeds(
    evokeds_dict,
    picks=right_ant,
    axes=axes[0, 1],
    colors=['red', 'red','blue', 'blue'], linestyles=['-', '--', '-', '--'],
    show=False,
    ylim=ylim
)
axes[0, 1].set_title("Right Anterior",size=20)

plt.show()

## Topographic difference plots

In [ ]:
# Difference between unrelated and identical primed conditions for each task
comp_dif = mne.combine_evoked([unrel_comp, ident_comp], weights = [1, -1])
prod_dif = mne.combine_evoked([unrel_prod, ident_prod], weights = [1,-1])

times = np.arange(0.35, 0.5, 0.025)
fig = comp_dif.plot_topomap(times, ch_type="mag", show=False, vlim=(-45, 45))
plt.suptitle('Comprehension Unprimed-Primed', fontsize=20, color='blue')
plt.show()

times = np.arange(0.35, 0.5, 0.025)
fig = prod_dif.plot_topomap(times, ch_type="mag", show=False, vlim=(-45,45))
plt.suptitle('Production Unprimed-Primed', fontsize=20, color='red')
plt.show()

# Obtain STCs (Source Localization!)

Inferring source space activity from  sensor data requires that you have **FreeSurfer** installed, and that you carried out coregistration. Coregistration will produce an anatomical model of the subject's brain based on the digitization as well as yielding a transformation matrix.

To install FreeSurfer, follow the instructions on [this page](https://surfer.nmr.mgh.harvard.edu/fswiki/DownloadAndInstall).

In this tutorial, coregistration has been done for you. All the results can be found in the Tutorial/freesurfer/R3289 folder. You can follow the instructions in [this document](https://docs.google.com/document/d/16D3C3NQU8hzEc976RHI2Z3RJRwI9mlbs-yQkQ6N0To8/edit?usp=sharing) to do it yourself.

In [ ]:
subjects_dir = f"{ROOT}Tutorial/freesurfer/"
directory = f"{ROOT}Tutorial/{sub}/"

We will set some generic parameters for source localization. More information can be found here.

In [ ]:
conductivity = (0.3,) # single layer conductivity
baseline_start = -300 #in milliseconds
baseline_end = 0
baseline = (None,0)
snr = 3.0
method = "dSPM"

Now, we will create the anatomical model of our subject. BEM model documentation can be found [here](https://mne.tools/stable/generated/mne.make_bem_model.html), and source space documentation can be found [here](https://mne.tools/stable/generated/mne.setup_source_space.html).

In [ ]:
subject = sub
model = mne.make_bem_model(subject=subject, ico=4, conductivity=conductivity, subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)
src = mne.setup_source_space(subject, spacing="oct6", add_dist="patch", subjects_dir=subjects_dir)

We will now compute the noise covariance using the baseline interval, which spans from the beginning of the evoked timescale (in this case, -300ms), to the image/word onset (0ms).

It is common practice to use the [baseline interval for noise covariance](https://mne.tools/stable/auto_tutorials/forward/90_compute_covariance.html) in evoked experiments, though one could also use the empty room file.

The MNE function does not automatically calculate the noise covariance rank accurately, in my experience. This could be related to the fact that the automatic rank computation does not do a good job taking IC removal during ICA into account. [Here](https://github.com/mne-tools/mne-python/issues/7727) and [here](https://mne.discourse.group/t/specify-noise-covariance-rank-in-mne-python/888/5) are threads that discusses the issue.

My approach is to set the rank manually. Basically, you want to find a numerical value that sets the red dotted line right at the edge of the "shelf" of the curve of noise plotted over eigenvalues.
**Below, change X in {'mag': X} to a value that sets the vertical dotted line at the correct position on the curve.**

Let me know if you find a better, automatic way to do this step.

In [ ]:
rank = {'mag': 149}
noise_cov_comp = mne.compute_covariance(epochs_comp, tmin=baseline_start, tmax=baseline_end,
                                        method=["shrunk"], verbose=True)
mne.viz.plot_cov(noise_cov_comp, epochs_comp.info)

Once the rank value looks good, you can compute the forward solution.

In [ ]:
trans = input_dir + sub + '-trans.fif'
fname = output_dir + '/' + sub + '_comp_preproc-raw.fif'

# Compute forward solution
fwd_comp = mne.make_forward_solution(
    fname,
    trans=trans,
    src=src,
    bem=bem,
    meg=True,
    eeg=False,
    mindist=5.0,
    ignore_ref=True
)

Then, you can compute the inverse solution

In [ ]:
lambda2 = 1.0 / snr ** 2

# Calculate inverse operators
inverse_ident_comp = make_inverse_operator(ident_comp.info, fwd_comp, noise_cov_comp, loose=0.2, depth=0.8)
inverse_unrel_comp = make_inverse_operator(unrel_comp.info, fwd_comp, noise_cov_comp, loose=0.2, depth=0.8)

Then, you can imply the inverse operator to obtain and save your source time courses.

In [ ]:
# Calculate STCs using evoked data, inverse operators from above
stc_ident_comp = apply_inverse(ident_comp, inverse_ident_comp, lambda2, method=method, pick_ori="normal")
stc_unrel_comp = apply_inverse(unrel_comp, inverse_unrel_comp, lambda2, method=method, pick_ori="normal")

A good inverse solution will explain a large amount of the variance in the original data. If your solution explains <65% of the variance, you should try to find a better value for rank, or try to identify if anything could be contributing to a low SNR.

If the inverse solution looks good, you can save your files.

In [ ]:
fname_ident_comp = output_dir + sub + '_ident_comp'
stc_ident_comp.save(fname_ident_comp, ftype='stc', overwrite=True)
fname_unrel_comp = output_dir + sub + '_unrel_comp'
stc_unrel_comp.save(fname_unrel_comp, ftype='stc', overwrite=True)

Do the exact same for production.

In [ ]:
rank = {'mag': 148}
noise_cov_prod = mne.compute_covariance(epochs_prod, tmin=baseline_start, tmax=baseline_end, method=["shrunk", "empirical"], rank=rank, verbose=True)
mne.viz.plot_cov(noise_cov_prod, epochs_prod.info)

In [ ]:
fname = output_dir + '/' + sub + '_prod_preproc-raw.fif'
trans = input_dir + sub + '-trans.fif'

# Compute forward solution
fwd_prod = mne.make_forward_solution(
    fname,
    trans=trans,
    src=src,
    bem=bem,
    meg=True,
    eeg=False,
    mindist=5.0,
    ignore_ref=True
)
lambda2 = 1.0 / snr ** 2

# Calculate inverse operators
inverse_ident_prod = make_inverse_operator(ident_prod.info, fwd_prod, noise_cov_prod, loose=0.2, depth=0.8)
inverse_unrel_prod = make_inverse_operator(unrel_prod.info, fwd_prod, noise_cov_prod, loose=0.2, depth=0.8)

# Calculate STCs using evoked data, inverse operators from above
stc_ident_prod = apply_inverse(ident_prod, inverse_ident_prod, lambda2, method=method, pick_ori="normal")
stc_unrel_prod = apply_inverse(unrel_prod, inverse_unrel_prod, lambda2, method=method, pick_ori="normal")

fname_ident_comp = directory + sub + '_ident_prod'
stc_ident_prod.save(fname_ident_comp, ftype='stc', overwrite=True)
fname_unrel_comp = directory + sub + '_unrel_prod'
stc_unrel_prod.save(fname_unrel_comp, ftype='stc', overwrite=True)

# Plot STCs

## Individual Conditions

In [ ]:
brain = stc_ident_prod.plot(
    subjects_dir=subjects_dir,
    hemi='lh',
    clim=dict(kind="value", lims=[0,2,4]),
    smoothing_steps=7,
    initial_time=0.4,
    background='white',
    show_traces=False,
    title='Production Repetition Primed')

In [ ]:
brain = stc_unrel_prod.plot(
    subjects_dir=subjects_dir,
    hemi='lh',
    clim=dict(kind="value", lims=[0,2,4]),
    smoothing_steps=7,
    initial_time=0.4,
    background='white',
    show_traces=False,
    title='Production Unrelated Primed')

In [ ]:
brain = stc_ident_comp.plot(
    subjects_dir=subjects_dir,
    hemi='lh',
    clim=dict(kind="value", lims=[0,2,4]),
    smoothing_steps=7,
    initial_time=0.4,
    background='white',
    show_traces=False,
    title='Comprehension Repetition Primed')

In [ ]:
brain = stc_unrel_comp.plot(
    subjects_dir=subjects_dir,
    hemi='lh',
    clim=dict(kind="value", lims=[0,2,4]),
    smoothing_steps=7,
    initial_time=0.4,
    background='white',
    show_traces=False,
    title='Comprehension Unrelated Primed')

## Differences between conditions

In [ ]:
stc_dif_prod = stc_unrel_prod - stc_ident_prod
stc_dif_comp = stc_unrel_comp - stc_ident_comp

In [ ]:
brain = stc_dif_prod.plot(
    subjects_dir=subjects_dir,
    hemi='lh',
    clim=dict(kind="value", lims=[0,2,4]),
    smoothing_steps=7,
    initial_time=0.4,
    background='white',
    show_traces=False,
    title='Production Difference')

In [ ]:
brain = stc_dif_comp.plot(
    subjects_dir=subjects_dir,
    hemi='lh',
    clim=dict(kind="value", lims=[0,2,4]),
    smoothing_steps=7,
    initial_time=0.4,
    background='white',
    show_traces=False,
    title='Comprehension Difference')